# AAI540 - Module 3 Assignment

Victor Hugo Germano

In [2]:
import boto3
import sagemaker

original_boto3_version = boto3.__version__
%pip install 'boto3>1.17.21'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

In [4]:
# S3 definitino
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = "sagemaker-featurestore-housing"

print(default_s3_bucket_name)

sagemaker-us-east-1-440542329720


In [5]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

arn:aws:iam::440542329720:role/LabRole


## Dataset 

In [6]:
# define s3 address

s3_private_path_data = "s3://{}/housing-m3/csv".format(default_s3_bucket_name)


print(s3_private_path_data)
%store s3_private_path_data


housing_local_file_path = "housing.csv"
housing_s3_object_key = 'housing-m3.csv'


gmaps_local_file_path = "housing_gmaps_data_raw.csv"
gmaps_s3_object_key = 'gmaps-m3.csv'

%store housing_local_file_path
%store gmaps_local_file_path
%store housing_s3_object_key
%store gmaps_s3_object_key

s3://sagemaker-us-east-1-440542329720/housing-m3/csv
Stored 's3_private_path_data' (str)
Stored 'housing_local_file_path' (str)
Stored 'gmaps_local_file_path' (str)
Stored 'housing_s3_object_key' (str)
Stored 'gmaps_s3_object_key' (str)


In [7]:


# copy csv to s3
!aws s3 cp $housing_local_file_path $s3_private_path_data/$housing_s3_object_key 
!aws s3 cp $gmaps_local_file_path $s3_private_path_data/$gmaps_s3_object_key 

print(s3_private_path_data)
!aws s3 ls $s3_private_path_data/
    

upload: ./housing.csv to s3://sagemaker-us-east-1-440542329720/housing-m3/csv/housing-m3.csv
upload: ./housing_gmaps_data_raw.csv to s3://sagemaker-us-east-1-440542329720/housing-m3/csv/gmaps-m3.csv
s3://sagemaker-us-east-1-440542329720/housing-m3/csv
2026-01-26 16:36:18    2057894 gmaps-m3.csv
2026-01-26 16:36:17    1423529 housing-m3.csv


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io


s3_client = boto3.client("s3", region_name=region)

housing_data_object = s3_client.get_object(
    Bucket=default_s3_bucket_name, Key=f"housing-m3/csv/{housing_s3_object_key}"
)
gmaps_data_object = s3_client.get_object(
    Bucket=default_s3_bucket_name, Key=f"housing-m3/csv/{gmaps_s3_object_key}"
)

# define index as their position
housing_data = pd.read_csv(io.BytesIO(housing_data_object["Body"].read()))
housing_data["index"] = housing_data.index
gmaps_data = pd.read_csv(io.BytesIO(gmaps_data_object["Body"].read()))
gmaps_data["index"] = gmaps_data.index


In [14]:
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,index
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,2
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,4


In [15]:
gmaps_data.head()

,street_number,route,locality-political,administrative_area_level_2-political,administrative_area_level_1-political,country-political,postal_code,address,longitude,latitude,...,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest,index
0,3130,Grizzly Peak Boulevard,Berkeley,Alameda County,California,United States,94705.0,"3130 Grizzly Peak Blvd, Berkeley, CA 94705, USA",-122.23,37.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2005,Tunnel Road,Oakland,Alameda County,California,United States,94611.0,"2005 Tunnel Rd, Oakland, CA 94611, USA",-122.22,37.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,6886,Chabot Road,Oakland,Alameda County,California,United States,94618.0,"6886 Chabot Rd, Oakland, CA 94618, USA",-122.24,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,6365,Florio Street,Oakland,Alameda County,California,United States,94618.0,"6365 Florio St, Oakland, CA 94618, USA",-122.25,37.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,5407,Bryant Avenue,Oakland,Alameda County,California,United States,94618.0,"5407 Bryant Ave, Oakland, CA 94618, USA",-122.25,37.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


## Feature engineering

In [16]:
# Merge housing data with gmaps data to get neighborhood information
merged_data = pd.merge(
    housing_data,
    gmaps_data,
    on="index",
    how='left'
)

print(f"Original housing data shape: {housing_data.shape}")
print(f"Merged data shape: {merged_data.shape}")
merged_data.head()

Original housing data shape: (20640, 11)
Merged data shape: (20640, 41)


,longitude_x,latitude_x,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,...,establishment-natural_feature,airport-establishment-point_of_interest,political-sublocality-sublocality_level_1,administrative_area_level_3-political,post_box,establishment-light_rail_station-point_of_interest-transit_station,establishment-point_of_interest,aquarium-establishment-park-point_of_interest-tourist_attraction-zoo,campground-establishment-lodging-park-point_of_interest-rv_park-tourist_attraction,cemetery-establishment-park-point_of_interest
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Create one-hot encoded columns for ocean_proximity
ocean_proximity_dummies = pd.get_dummies(merged_data['ocean_proximity'], prefix='', prefix_sep='', dtype=int)

# Ensure all expected columns exist (in case some categories are missing)
expected_columns = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
for col in expected_columns:
    if col not in ocean_proximity_dummies.columns:
        ocean_proximity_dummies[col] = 0

# Rename columns to match requirements (lowercase with underscores)
ocean_proximity_dummies = ocean_proximity_dummies.rename(columns={
    '<1H OCEAN': 'less_than_1h_ocean',
    'INLAND': 'inland',
    'ISLAND': 'island',
    'NEAR BAY': 'near_bay',
    'NEAR OCEAN': 'near_ocean'
})

# Add one-hot encoded columns to merged data
merged_data = pd.concat([merged_data, ocean_proximity_dummies], axis=1)

print("One-hot encoded ocean_proximity columns created:")
print(ocean_proximity_dummies.head())

One-hot encoded ocean_proximity columns created:
   less_than_1h_ocean  inland  island  near_bay  near_ocean
0                   0       0       0         1           0
1                   0       0       0         1           0
2                   0       0       0         1           0
3                   0       0       0         1           0
4                   0       0       0         1           0


In [20]:
#Calculate bedrooms per household with imputation
# calculate average bedrooms per household by postal code for imputation
merged_data['bedrooms_per_household'] = merged_data['total_bedrooms'] / merged_data['households']

# Calculate average bedrooms per household by postal code
postal_code_avg = merged_data.groupby('postal_code')['bedrooms_per_household'].transform('mean')

# Impute missing values with postal code average
merged_data['bedrooms_per_household'] = merged_data['bedrooms_per_household'].fillna(postal_code_avg)

# If still missing (no postal code data), use global average
global_avg = merged_data['bedrooms_per_household'].mean()
merged_data['bedrooms_per_household'] = merged_data['bedrooms_per_household'].fillna(global_avg)

print(f"Bedrooms per household calculated.")
print(f"Missing values after imputation: {merged_data['bedrooms_per_household'].isna().sum()}")
merged_data[['total_bedrooms', 'households', 'bedrooms_per_household', 'postal_code']].head()

Bedrooms per household calculated.
Missing values after imputation: 0


,total_bedrooms,households,bedrooms_per_household,postal_code
0,129.0,126.0,1.023810,94705.0
1,1106.0,1138.0,0.971880,94611.0
2,190.0,177.0,1.073446,94618.0
3,235.0,219.0,1.073059,94618.0
4,280.0,259.0,1.081081,94618.0


In [21]:
def discretize_age(age):
    """Discretize age into 10-year buckets: 0-9, 10-19, 20-29, etc."""
    if pd.isna(age):
        return None
    bucket_start = int(age // 10) * 10
    bucket_end = bucket_start + 9
    return f"{bucket_start}-{bucket_end}"

# Test the function
print("Age discretization examples:")
for age in [5, 15, 25, 35, 45]:
    print(f"Age {age} -> {discretize_age(age)}")

Age discretization examples:
Age 5 -> 0-9
Age 15 -> 10-19
Age 25 -> 20-29
Age 35 -> 30-39
Age 45 -> 40-49


In [23]:
# Group by neighborhood-political and aggregate features

# Using locality-political as aggregate features

# Filter out rows with missing locality-political
merged_data_with_neighborhood = merged_data[merged_data['locality-political'].notna()].copy()

neighborhood_features = merged_data_with_neighborhood.groupby('locality-political').agg({
    'median_house_value': lambda x: min(x.mean(), 500000),
    
    # Median house age: average across neighborhood
    'median_house_age': 'mean',
    
    # Total households: average across neighborhood, rounded up
    'households': lambda x: int(np.ceil(x.mean())),
    
    # Bedrooms per household: average across neighborhood
    'bedrooms_per_household': 'mean',
    
    # One-hot encoded ocean proximity: take max (1 if any house in neighborhood has it)
    'less_than_1h_ocean': 'max',
    'inland': 'max',
    'island': 'max',
    'near_bay': 'max',
    'near_ocean': 'max'
}).reset_index()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 # Filter out rows with missing locality-political                                           │
│    6 merged_data_with_neighborhood = merged_data[merged_data['locality-political'].notna()].c    │
│    7                                                                                             │
│ ❱  8 neighborhood_features = merged_data_with_neighborhood.groupby('locality-political').agg(    │
│    9 │   # Median house value: average across neighborhood, capped at 500,000                    │
│   10 │   'median_house_value': lambda x: min(x.mean(), 500000),                                  │
│   11                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pandas/core/groupby/generic.py:1432 in aggregate         │
│                                                                                                  │
│   1429 │   │   │   kwargs["engine_kwargs"] = engine_kwargs                                       │
│   1430 │   │                                                                                     │
│   1431 │   │   op = GroupByApply(self, func, args=args, kwargs=kwargs)                           │
│ ❱ 1432 │   │   result = op.agg()                                                                 │
│   1433 │   │   if not is_dict_like(func) and result is not None:                                 │
│   1434 │   │   │   # GH #52849                                                                   │
│   1435 │   │   │   if not self.as_index and is_list_like(func):                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pandas/core/apply.py:190 in agg                          │
│                                                                                                  │
│    187 │   │   │   return self.apply_str()                                                       │
│    188 │   │                                                                                     │
│    189 │   │   if is_dict_like(func):                                                            │
│ ❱  190 │   │   │   return self.agg_dict_like()                                                   │
│    191 │   │   elif is_list_like(func):                                                          │
│    192 │   │   │   # we require a list, but not a 'str'                                          │
│    193 │   │   │   return self.agg_list_like()                                                   │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/pandas/core/apply.py:423 in agg_dict_like                │
│                                                                                                  │
│    420 │   │   -------                                                                           │
│    421 │   │   Result of aggregation.                                                            │
│    422 │   │   """                                                                               │
│ ❱  423 │   │   return self.agg_or_apply_dict_like(op_name="agg")                                 │
│    424 │                                                                                         │
│    425 │   def compute_dict_like(                                                                │
│    426 │   │   self,                                       